# English Topic Modeling — Harry Potter Fanfiction

In these lessons, we're learning about a text analysis method called *topic modeling*. This method will help us identify the main topics or discourses within a collection of texts a single text that has been separated into smaller text chunks.

---

## Dataset

### Harry Potter Fanfiction (English)

In this particular lesson, we're going to use [Little MALLET Wrapper](https://github.com/maria-antoniak/little-mallet-wrapper), a Python wrapper for [MALLET](http://mallet.cs.umass.edu/topics.php), to topic model a CSV file with fanfiction stories from the website [Archive of Our Own (AO3)](https://archiveofourown.org/).

___

<div class="admonition attention" name="html-admonition" style="background: lightyellow;padding: 10px">  
<p class="title">Attention</p>  
    
<p>If you're working in this Jupyter notebook on your own computer, you'll need to have both the Java Development Kit and MALLET pre-installed. For set up instructions, please see <a href="http://melaniewalsh.github.io/Intro-Cultural-Analytics/Text-Analysis/Topic-Modeling-Set-Up.html">the previous lesson<a/>.  </p>
    
If you're working in this Jupyter notebook in the cloud via Binder/JupyterHub, then the Java Development Kit and Mallet will already be installed. You're good to go!  
     
</div>  

## Set MALLET Path

Since Little MALLET Wrapper is a Python package built around MALLET, we first need to tell it where the bigger, Java-based MALLET lives.

We're going to make a variable called `path_to_mallet` and assign it the file path of our MALLET program. We need to point it, specifically, to the "mallet" file inside the "bin" folder inside the "mallet-2.0.8" folder. 

In [ ]:
path_to_mallet = '../../mallet-2.0.8/bin/mallet'

If MALLET is located in another directory, then set your `path_to_mallet` to that file path.

## Install Packages

In [ ]:
#!pip install little_mallet_wrapper
#!pip install seaborn
#To install the most updated version of little_mallet_wrapper:
#!!pip install git+https://github.com/maria-antoniak/little-mallet-wrapper.git

In [ ]:
!pip install scipy

## Import Packages

In [ ]:
import little_mallet_wrapper
import seaborn
import pandas as pd
import random
from pathlib import Path
pd.options.display.max_colwidth = 100

## Get Training Data From CSV File

In [ ]:
hp_df_english = pd.read_csv("harry_potter_english_stories_first_chaps.csv")

In [ ]:
hp_df_english

Drop rows with no story text

In [ ]:
hp_df_english = hp_df_english.dropna(subset=['body'])

### Pre-Process Texts

`little_mallet_wrapper.process_string(text, numbers='remove')`

Next we're going to process our texts with the function `little_mallet_wrapper.process_string()`. This function will take every individual post, transform all the text to lowercase as well as remove stopwords, punctuation, and numbers, and then add the processed text to our master list `training_data`.

In [ ]:
training_data = []
for text in hp_df_english['body']:
    processed_text = little_mallet_wrapper.process_string(text, numbers='remove)                                                      #stop_words_extra=spanish_stop_words)
    training_data.append(processed_text)

In [ ]:
original_texts = [text for text in hp_df_english['body']]

In [ ]:
hp_english_titles = [title for title in hp_df_english['title']]

### Get Dataset Statistics

We can get training data summary statisitcs by using the funciton `little_mallet_wrapper.print_dataset_stats()`.

In [ ]:
little_mallet_wrapper.print_dataset_stats(training_data)

## Training the Topic Model

We need to make a variable `num_topics` and assign it the number of topics we want returned. Then we're going to set a file path where we want all our MALLET topic modeling data to be dumped.

In [ ]:
# your choice of topics
num_topics = 50

path_to_mallet = '../../mallet-2.0.8/bin/mallet'

#No need to change anything below here
training_data = training_data

#Set output directory
output_directory_path = 'topic-model-output/hp-fanfiction/english-firstchaps'

#Create output directory
Path(f"{output_directory_path}").mkdir(parents=True, exist_ok=True)

#Create output files
path_to_training_data           = f"{output_directory_path}/training.txt"
path_to_formatted_training_data = f"{output_directory_path}/mallet.training"
path_to_model                   = f"{output_directory_path}/mallet.model.{str(num_topics)}"
path_to_topic_keys              = f"{output_directory_path}/mallet.topic_keys.{str(num_topics)}"
path_to_topic_distributions     = f"{output_directory_path}/mallet.topic_distributions.{str(num_topics)}"

### Train Topic Model

Then we're going to train our topic model with `little_mallet_wrapper.quick_train_topic_model()`.

In [ ]:
#little_mallet_wrapper.quick_train_topic_model(path_to_mallet,
#                                            output_directory_path,
#                                             num_topics,
#                                             training_data)

When the topic model finishes, it will output your results to your `output_directory_path`.

## Display Topics and Top Words

To examine the topics that the topic model extracted from the training data, run the cell below. This code uses the `little_mallet_wrapper.load_topic_keys()` function to read and process the MALLET topic model output.

In [ ]:
topics = little_mallet_wrapper.load_topic_keys(path_to_topic_keys)

for topic_number, topic in enumerate(topics):
    
    print(f"✨Topic {topic_number}✨\n\n{topic}\n")

## Load Topic Distributions

MALLET also calculates the likely mixture of these topics for every single document in the corpus. This mixture is really a probability distribution, that is, the probability that each topic exists in the document. We can use these probability distributions to examine which of the above topics are strongly associated with which specific documents.

To get the topic distributions, we're going to use the `little_mallet_wrapper.load_topic_distributions()` function, which will read and process the MALLET topic model output.

In [ ]:
topic_distributions = little_mallet_wrapper.load_topic_distributions(path_to_topic_distributions)

In [ ]:
topic_distributions[0]

## Display Top Documents Per Topic

In [ ]:
from IPython.display import Markdown, display
import re

def make_md(string):
    """A function that transforms string data into Markdown
    so it can be nicely formatted with bolding and emojis
    """
    display(Markdown(str(string)))

def get_top_docs(docs, topic_distributions, topic_index=1, n=5, doc_length = 2000):
    
    """A function that shows the top documents for a given set of topic distributions
    and a specific topic number
    """
    
    sorted_data = sorted([(_distribution[topic_index], _document) for _distribution, _document in zip(topic_distributions, docs)], reverse=True)
    topic_words = topics[topic_index]
    make_md(f"### ✨Topic {topic_index}✨\n\n{topic_words}\n\n---")
    
    for probability, doc in sorted_data[:n]:
        # Make topic words bolded
        for word in topic_words:
            if word in doc.lower():
                doc = re.sub(f"\\b{word}\\b", f"**{word}**", doc, re.IGNORECASE)
        make_md(f'✨  \n**Topic Probability**: {probability}  \n**Document**: {doc[:doc_length]}\n\n')

# Work, Coffee, Tea

In [ ]:
get_top_docs(original_texts, topic_distributions, topic_index=30, n=10)

# Family

In [ ]:
get_top_docs(original_texts, topic_distributions, topic_index=13, n=5)

# Goblins

In [ ]:
get_top_docs(original_texts, topic_distributions, topic_index=9, n=5, doc_length = 4000)

In [ ]:
get_top_docs(hp_english_titles, topic_distributions, topic_index=9, n=5)

# Sex

In [ ]:
get_top_docs(original_texts, topic_distributions, topic_index=41, n=5, doc_length = 4000)